In [1]:
from statsmodels.stats.power import TTestIndPower
import math

In [2]:
def ab_test_duration(sigma, delta, alpha=0.05, power=0.8, daily_users=50, alternative='two-sided'):
    """
    sigma: стандартне відхилення метрики
    delta: мінімальна різниця, яку хочемо виявити
    alpha: рівень значущості (тип I помилка)
    power: потужність тесту (1 - beta)
    daily_users: середня кількість користувачів на день
    alternative: тип альтернативної гіпотези ('two-sided', 'larger', 'smaller')
        'two-sided' — двосторонній тест. Перевіряє, чи є будь-яка різниця між групами (вище чи нижче).
        'larger' — односторонній тест, перевіряє, чи нова група більше за контроль.
        'smaller' — односторонній тест, перевіряє, чи нова група менше за контроль.
    """
    analysis = TTestIndPower()
    
    # effect_size = різниця / стандартне відхилення
    effect_size = delta / sigma
    
    n_per_group = analysis.solve_power(effect_size=effect_size,
                                       alpha=alpha,
                                       power=power,
                                       alternative=alternative)
    
    n_per_group = math.ceil(n_per_group)
    days_needed = math.ceil(n_per_group / daily_users)
    
    return n_per_group, days_needed

In [3]:
sigma = 20          # стандартне відхилення депозиту
delta = 5           # мінімальна різниця, яку хочемо виявити
daily_users = 50    # користувачів на день

n, days = ab_test_duration(sigma, delta, daily_users=daily_users)
print(f"Користувачів на групу: {n}")
print(f"Тривалість тесту (днів): {days}")

Користувачів на групу: 253
Тривалість тесту (днів): 6
